<a href="https://colab.research.google.com/github/NadAAaHH/Arabic-Error-Checker/blob/main/Allam_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

In [ ]:
!pip install --upgrade pip setuptools wheel

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [18]:
import os
import pandas as pd
import getpass


# langchain
import matplotlib.pyplot as plt
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain_ibm import WatsonxEmbeddings
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain


# ibm_watsonx_ai
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# 1. Loading CSVs and chunking with LangChain



In [19]:
# Simple method - Split by pages
loader = CSVLoader("/content/الأخطاء اللغوية الشائعة (تنظيف).csv")
pages = loader.load_and_split()

print(pages[1])

# SKIP TO STEP 2 IF YOU'RE USING THIS METHOD
chunks = pages

page_content='Error: أَجِبْ عَلى الأسئلة الآتية.
Correction: أَجِبْ عَن الأسئلة الآتية.
Explanation: " أجب عن " بدلاً من " أجب على " لأن الإجابة عن الأسئلة تعني تحديد إجابتها المطلوبة، ولكن "على الأسئلة تعني ترك الأسئلة والإجابة عن غيرها، خاصةً أن حرفَ الجر "على" يعني المُجاوزة وترك الشيء كما هو معروف. وهذا الخطأ يتكرر كثيراً في الاختبارات، لذا يجب أن يتنبه الَّذِين يَضَعُونَ الاختبارات لهذا الأمر. قال تعالى : ومَنْ يَبْخَلْ فإنّما يبخلُ عَنْ نَفْسِهِ" (?) فاستخدم الحق سبحانه يبخلُ" عن" ولم يستخدم يبخل على والفرق واضح بين الاثنين بناءً على ما تقدم.
:' metadata={'source': '/content/الأخطاء اللغوية الشائعة (تنظيف).csv', 'row': 1}


# 2. Embed text and store embeddings

### Seting up connection with watsonx.ai and get environment Variables

Get watsonx URL

In [36]:
try:
    os.environ["WATSONX_URL"] = wxa_url
except KeyError:
    wxa_url = getpass.getpass("Please enter your watsonx.ai URL domain (hit enter): ")

Get watsonx API key

In [31]:
try:
     os.environ["WATSONX_APIKEY"] = wxa_api_key
except KeyError:
    wxa_api_key = getpass.getpass("Please enter your watsonx.ai API key (hit enter): ")

Get watsonx Project ID

In [32]:
try:
    os.environ["WATSONX_PROJECT_ID"] = wxa_project_id
except KeyError:
    wxa_project_id = getpass.getpass("Please enter your watsonx.ai Project ID (hit enter): ")

Please enter your watsonx.ai Project ID (hit enter): ··········


### Loading the embedding model

In [42]:
# adjust model parameters
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

In [46]:
multilingual_e5_embedding = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    params=embed_params,
    url=wxa_url,
    project_id=wxa_project_id
)

### Initilaize FIASS vector database

In [47]:
# Create vector database
db = FAISS.from_documents(chunks, multilingual_e5_embedding)

# 3. Setup retrieval function

In [48]:
# Check that similarity search is working
query = "قام المعلمين برسم المنظر."
docs = db.similarity_search(query, k=10)
for i in docs:
  print(i)
  print("-------------")

page_content='Error: المعلم كف لتدريس المبحث .
Correction: المعلم كفء لتدريس المبحث.
Explanation: لا يصح وضع الهمزة على حرف الفاء في كلمة كفء؛ لأنَّ الهمزة المتطرفة المسبوقة بحرفِ ساكن تُكْتَبُ على السطر، ولا يصح وضعها على الفاء أو على الألف أو على الياء ولكن إذا قلت: هو معلم كَفِيّ فهذا جائز ، وجمعه أكفياء أما هو معلم كفء وجمعه : أكْفاء، نقولُ : معلمون أكفاء. وهكذا. والخلاصة أنّ "أَكْفَاء" مفردها "كُفء". وأن أكفباء مفردها "كفي". وأنّ "أكفاء" مفردها "كَفيف"
:' metadata={'source': '/content/الأخطاء اللغوية الشائعة (تنظيف).csv', 'row': 28}
-------------
page_content='Error: المعلمون أكفاء.
Correction: المعلمون أكفاء.
Explanation: لا يصح تشديد الفاء في جمع كلمة كفء فإذا شددت أصبحت بمعنى آخر ، والمعلمون الأكفاء هم القادرون الفاعلون، أما كلمة "أكفاء" بالتشديد فتعني أنهم من ذوي الحاجات الخاصة الذين فقدوا البصر، وهذا معنى بعيد عن الكفاءة التي يُوصَفُ بها المعلمون. 
وتقول : فلان ذو كفاءة. 
وتقول: آخر ذو كفاية. 
ومعنى الأول : أنه كف ولكن له نظراء وأمثال . 
والثاني: أنه كفء وليس له نظراء وأمثال


## Loading ALLAM model

In [49]:
# adjust model parameters
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500
}

In [50]:
allam_llm = WatsonxLLM(
    model_id="sdaia/allam-1-13b-instruct",
    params=parameters,
    url=wxa_url,
    project_id=wxa_project_id
)

### Test result from ALLAM model directly

In [51]:
# define input query
query = "صحح الخطأ في الجملة التالية: قام المعلمين برسم المنظر."

In [52]:
response_1 = allam_llm(query)

<ipython-input-52-104bc6715523>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response_1 = allam_llm(query)


In [53]:
print("Query: " + query)
print("Response: " + response_1)

Query: صحح الخطأ في الجملة التالية: قام المعلمين برسم المنظر.
Response:  الإجابة الصحيحة : قام المعلمون برسم المنظر . الخطأ هنا هو استخدام صيغة المفرد "معلم " بدلاً من صيغة الجمع "معلمون " ، حيث يجب أن تكون الجملة بصيغة الجمع لأن الفاعل هو "المعلمون " وليس المعلم الواحد . 


### Create QA chain to integrate similarity search with input queries (answer query from knowledge base)

In [55]:
chain = load_qa_chain(allam_llm, chain_type="stuff")
response_2 = chain.run(input_documents=docs, question=query)

<ipython-input-55-f238acb34f13>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(allam_llm, chain_type="stuff")
<ipython-input-55-f238acb34f13>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response_2 = chain.run(input_documents=docs, question=query)


In [56]:
print("Query: " + query)
print("Response:" + response_2)

Query: صحح الخطأ في الجملة التالية: قام المعلمين برسم المنظر.
Response: قام المعلمون برسم المنظر.
Explanation: في الجملة الأصلية "قام المعلمين برسم المنظر" يوجد خطأ في استخدام جمع المذكر السالم "المعلمين". الصيغة الصحيحة يجب أن تكون "قام المعلمون برسم المنظر". جمع المذكر السالم يرفع بالواو وينصب ويجر بالياء، لذا يجب تغيير "المعلمين" إلى "المعلمون" ليصبح الجملة صحيحة نحوياً. 


### Elasticsearch method

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

emb_func = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
<ipython-input-1-a996f72d8854>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  emb_func = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# Defining model
model_id = ModelTypes.FLAN_UL2

In [ ]:
# Defining the model parameters
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

In [ ]:
# Initialize the Model class

model = Model(
    model_id="sdaia/allam-1-13b-instruct",
    credentials=Credentials(
        api_key = "9L8TsBVh597KH2qocdFOgBOFhwL93Rt-iayij5EYFyoM",
        url = "https://eu-de.ml.cloud.ibm.com"),
    project_id="64d518b6-f02e-4572-be32-cb1ba20cfd7a"
    )

### Basic information how to connect to Elasticsearch


In [ ]:
try:
    esuser = os.environ["ESUSER"]
except KeyError:
    esuser = input("Please enter your Elasticsearch user name (hit enter): ")
try:
    espassword = os.environ["ESPASSWORD"]
except KeyError:
    espassword = getpass.getpass("Please enter your Elasticsearch password (hit enter): ")
try:
    eshost = os.environ["ESHOST"]
except KeyError:
    eshost = input("Please enter your Elasticsearch hostname (hit enter): ")
try:
    esport = os.environ["ESPORT"]
except KeyError:
    esport = input("Please enter your Elasticsearch port number (hit enter): ")

Please enter your Elasticsearch user name (hit enter): 


NameError: name 'getpass' is not defined

In [ ]:
# Load the .env file
_ = load_dotenv(find_dotenv())

# Access the environment variables
api_key = os.getenv("IBM_WATSONX_API_KEY")

print(f"API Key: {api_key}")